# 1 - Introduction

Welcome to the Playground Season 4, Episode 1! A new year, a new set of challenges! In this particular challenge, we are tasked with predicting whether a user will close their account, or keep it open. Closing an account is known as _churn_. As with many other challenges we've seen previously, the metric we will be using is AUC ROC (Area Under the ROC Curve). The AUC ROC gives us a single metric that tells us how well our classifier is performing, regardless of what our classification threshold is. We'll explore this more in depth a bit later, along with ways we can use ranking to improve our predictions. For now, let's dig into the data!

# 1.1 - Initial Dataset Impressions

Let's look at the memory and disk footprints first, as this can sometimes be a limiting factor on what we can do.

| Dataset | Size on Disk | Size in Memory | # of Rows | # of Cols |
| ------- | ------------ | -------------- | --------- | --------- |
| `train` | 7.8 MB       | 45.6 MB        | 165,034   | 14        |
| `test`  | 12.0 MB      | 21.7 MB        | 110,023   | 13        |

As we can see, there is a healthy number of rows to work with, suggesting a lot of data for training. This also suggests that we may have better agreement between public and private leaderboards, depending on whether the dataset distributions match each other between train and test. Finally, our memory footprint is not terribly large, although we may start to run into memory pressure issues if we perform too much feature engineering or attempt to build large models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv("../input/playground-series-s4e1/train.csv")
test = pd.read_csv("../input/playground-series-s4e1/test.csv")

trainsize = train.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"train dataset memory usage: {trainsize:,.2f} MB")
train

In [ ]:
testsize = test.memory_usage(deep=True).sum() / (1024 ** 2)
print(f"test dataset memory usage: {testsize:,.2f} MB")
test

### Key Observations About Initial Dataset Impressions

* With a large number of samples to learn from, our results may be more stable between local CV, public LB, and private LB. 
* There are 11 different features to learn from:
    * From a rough overview, they appear to be a mixture of continuous features (`CreditScore`, `Balance`), categorical (`Tenure`, `Surname`, `Geography`), and boolean (`HasCrCard`, `IsActiveMember`). We will need to dig deeper to ensure this is indeed the case.

# 1.2 - Null Values

Let's explore the issue of missing values in the dataset to see if there are systemic problems with data representation.

In [ ]:
train["null_count"] = train.isnull().sum(axis=1)
counts = train.groupby("null_count")["id"].count().to_dict()
null_data = {"{} Null Value(s)".format(k) : v for k, v in counts.items() if k < 8}

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))

axs = axs.flatten()

_ = axs[0].pie(
    x=list(null_data.values()), 
    autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(null_data.values())/100, x),
    explode=[0.05] * len(null_data.keys()), 
    labels=null_data.keys(), 
    colors=sns.color_palette("Set2")[0:3],
)
_ = axs[0].set_title("Number of Null Values Per Row (Train)", fontsize=15)

test["null_count"] = test.isnull().sum(axis=1)
counts = test.groupby("null_count")["id"].count().to_dict()
null_data = {"{} Null Value(s)".format(k) : v for k, v in counts.items() if k < 8}

_ = axs[1].pie(
    x=list(null_data.values()), 
    autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(null_data.values())/100, x),
    explode=[0.05] * len(null_data.keys()), 
    labels=null_data.keys(), 
    colors=sns.color_palette("Set1")[0:3],
)
_ = axs[1].set_title("Number of Null Values Per Row (Test)", fontsize=15)

train = train.drop("null_count", axis=1)
test = test.drop("null_count", axis=1)

### Key Observations About Null Values

* No null values appear in the training or testing datasets.

# 1.3 - Train / Test Difference - Adversarial Validation

As a quick test, we should see how different the values are between train and test. To do so, we'll quickly perform a round of adversarial validation to see if a classifier can tell the two datasets apart. We'll use ROC AUC score to inform us of differences. If the two sets appear very similar, the classifier will not be able to tell them apart, and thus will have an ROC AUC score of 0.5. If they are easy to tell apart - and thus are dissimilar - then the ROC AUC score will approach 1. In order to handle categorical values, we'll use simple label encoding.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import roc_curve
from sklearn.preprocessing import LabelEncoder

train["origin"] = 0
test["origin"] = 1

combined = train.copy()
combined = pd.concat([combined, test]).reset_index(drop=True)

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    combined[feature] = le.fit_transform(combined[feature])
    
n_folds = 5
skf = KFold(n_splits=n_folds, random_state=2024, shuffle=True)
train_oof_preds = np.zeros((combined.shape[0],))
train_oof_probas = np.zeros((combined.shape[0],))

for fold, (train_index, test_index) in enumerate(skf.split(combined, combined["origin"])):
    print("-------> Fold {} <--------".format(fold + 1))
    x_train, x_valid = pd.DataFrame(combined.iloc[train_index]), pd.DataFrame(combined.iloc[test_index])
    y_train, y_valid = combined["origin"].iloc[train_index], combined["origin"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=2000,
        verbose=-1,  
        max_depth=3,
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(2000),
        ]
    )
    oof_preds = model.predict(x_valid_features[features])
    oof_probas = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    train_oof_probas[test_index] = oof_probas
    print(": AUC ROC = {}".format(roc_auc_score(y_valid, oof_probas)))
    
auc_vanilla = roc_auc_score(combined["origin"], train_oof_probas)
fpr, tpr, _ = roc_curve(combined["origin"], train_oof_probas)
print("--> Overall results for out of fold predictions")
print(": AUC ROC = {}".format(auc_vanilla))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

_ = sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets", ax=ax)
_ = sns.lineplot(x=fpr[::10], y=tpr[::10], ax=ax, label="Adversarial Validation Classifier")
_ = ax.set_title("ROC Curve", fontsize=15)
_ = ax.set_xlabel("False Positive Rate")
_ = ax.set_ylabel("True Positive Rate")

### Key Observations About Train / Test Difference - Adversarial Validation

* The trained classifier has an AUC ROC score of 0.504, which suggests that the training dataset and the testing dataset are very similar.

# 1.4 - Original Data - Adversarial Validation

One thing we can also look at is the original dataset, in this case the [Bank Customer Churn Prediction](https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction) dataset. We will perform an adversarial validation to see whether that dataset is similar to the competition dataset.

In [ ]:
original = pd.read_csv("../input/bank-customer-churn-prediction/Churn_Modelling.csv")

train["origin"] = 0
test["origin"] = 0
original["origin"] = 1

combined = train.copy()
combined = pd.concat([combined, test]).reset_index(drop=True)
combined = pd.concat([combined, original]).reset_index(drop=True)

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    combined[feature] = le.fit_transform(combined[feature])

n_folds = 5
skf = KFold(n_splits=n_folds, random_state=2024, shuffle=True)
train_oof_preds = np.zeros((combined.shape[0],))
train_oof_probas = np.zeros((combined.shape[0],))

for fold, (train_index, test_index) in enumerate(skf.split(combined, combined["origin"])):
    print("-------> Fold {} <--------".format(fold + 1))
    x_train, x_valid = pd.DataFrame(combined.iloc[train_index]), pd.DataFrame(combined.iloc[test_index])
    y_train, y_valid = combined["origin"].iloc[train_index], combined["origin"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=2000,
        verbose=-1,  
        max_depth=3,
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(2000),
        ]
    )
    oof_preds = model.predict(x_valid_features[features])
    oof_probas = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    train_oof_probas[test_index] = oof_probas
    print(": AUC ROC = {}".format(roc_auc_score(y_valid, oof_probas)))
    
auc_vanilla = roc_auc_score(combined["origin"], train_oof_probas)
fpr, tpr, _ = roc_curve(combined["origin"], train_oof_probas)
print("--> Overall results for out of fold predictions")
print(": AUC ROC = {}".format(auc_vanilla))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

_ = sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets", ax=ax)
_ = sns.lineplot(x=fpr[::10], y=tpr[::10], ax=ax, label="Adversarial Validation Classifier")
_ = ax.set_title("ROC Curve", fontsize=15)
_ = ax.set_xlabel("False Positive Rate")
_ = ax.set_ylabel("True Positive Rate")

The AUC ROC score of 0.778 suggests that the original dataset is quite different from the generated one. There may be a number of different reasons, but one obvious one may be due to the names that are used in both datasets. If we remove the names, we can check to see if the remaining data is similar or not.

In [ ]:
train["origin"] = 0
test["origin"] = 0
original["origin"] = 1

combined = train.copy()
combined = pd.concat([combined, test]).reset_index(drop=True)
combined = pd.concat([combined, original]).reset_index(drop=True)

features = [
    'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    combined[feature] = le.fit_transform(combined[feature])

n_folds = 5
skf = KFold(n_splits=n_folds, random_state=2024, shuffle=True)
train_oof_preds = np.zeros((combined.shape[0],))
train_oof_probas = np.zeros((combined.shape[0],))

for fold, (train_index, test_index) in enumerate(skf.split(combined, combined["origin"])):
    print("-------> Fold {} <--------".format(fold + 1))
    x_train, x_valid = pd.DataFrame(combined.iloc[train_index]), pd.DataFrame(combined.iloc[test_index])
    y_train, y_valid = combined["origin"].iloc[train_index], combined["origin"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=2000,
        verbose=-1,  
        max_depth=3,
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(2000),
        ]
    )
    oof_preds = model.predict(x_valid_features[features])
    oof_probas = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    train_oof_probas[test_index] = oof_probas
    print(": AUC ROC = {}".format(roc_auc_score(y_valid, oof_probas)))
    
auc_vanilla = roc_auc_score(combined["origin"], train_oof_probas)
fpr, tpr, _ = roc_curve(combined["origin"], train_oof_probas)
print("--> Overall results for out of fold predictions")
print(": AUC ROC = {}".format(auc_vanilla))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

_ = sns.lineplot(x=[0, 1], y=[0, 1], linestyle="--", label="Indistinguishable Datasets", ax=ax)
_ = sns.lineplot(x=fpr[::10], y=tpr[::10], ax=ax, label="Adversarial Validation Classifier")
_ = ax.set_title("ROC Curve", fontsize=15)
_ = ax.set_xlabel("False Positive Rate")
_ = ax.set_ylabel("True Positive Rate")

Slightly more similar, but still very distinct. Our categorical columns may still be contributing to easy to spot differences. Or, there may be other columns that are contributing to a very easy to spot difference between the original and the synthetic. We should dig in deeper to see if there are very obvious columns that contribute to similarity. If we can rule out easy-to-spot distinctions, we may be able to use the original dataset as a source of training data.

### Key Observations About Original Data - Adversarial Validation

* The classifier has an AUC ROC score of 0.778 - this is quite high, which suggests there are some easy-to-spot differences between the datasets.
    * Caution should be used when mixing data.
    * The `Surname` categorical column appears to be a distinguishing factor between datasets.
    * We should look to see if there are other big differences between datasets, so we can potentially use the original data to our advantage.

# 1.5 - Spearman Correlation

We should check to see if there is high correlation between our features. Spearman correlation does not make assumptions about distribution types or linearity. With Spearman correlation, we have values that range from -1 to +1. Values around either extreme end mean a negative or positive correlation respectively, while those around 0 mean no correlation exists.

In [ ]:
train_copy = train.copy()

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary', 'Exited'
]

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])

correlation_matrix = train_copy[features].corr(method="spearman")

from matplotlib.colors import SymLogNorm

f, ax = plt.subplots(figsize=(20, 20))
_ = sns.heatmap(
    correlation_matrix, 
    mask=np.triu(np.ones_like(correlation_matrix, dtype=bool)), 
    cmap=sns.diverging_palette(230, 20, as_cmap=True), 
    center=0,
    square=True, 
    linewidths=.1, 
    cbar=False,
    ax=ax,
    annot=True,
)
_ = ax.set_title("Spearman Correlation Matrix", fontsize=15)

### Key Observations about Spearman Correlation

* A strong negative correlation exists between the `NumOfProducts` and `Balance`. This is interesting, since it suggests that high balances are not tied to large number of products.
* A weakly negative correlation exists between `Age` and `NumOfProucts`.
* A strong positive correlation exists between `Geography` and `Balance`, suggesting there may be areas of the world that have higher balances.
* A negative correlation exists between `Surname` and `Balance`.
* In terms of correlations to target:
    * Strong positive correlations exist between `Age` and `Exited`, as well as `Balance` and `Exited`.
    * Strong negative correlations exist between `NumOfProducts` and `Exited`, `Gender` and `Exited`, as well as `IsActiveMember` and `Exited`.

# 1.7 - Statistical Breakdown

Let's take a closer look at some of the statistical properties of the continuous features. As a reminder, what we're looking for between the two sets is big differences between the min, max, and standard deviations for each continuous column. Differences there tell us if there are outliers that we need to deal with. Let's start with the training set.

In [ ]:
features = [
    'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train[features].describe().T.style.bar(subset=['mean'], color='#7BCC70')\
    .background_gradient(subset=['std'], cmap='Reds')\
    .background_gradient(subset=['50%'], cmap='coolwarm')

And for the testing set:

In [ ]:
features = [
    'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

test[features].describe().T.style.bar(subset=['mean'], color='#7BCC70')\
    .background_gradient(subset=['std'], cmap='Reds')\
    .background_gradient(subset=['50%'], cmap='coolwarm')

### Key Observations about Statistical Breakdown

* The min, max, and standard deviations between datasets suggest they are nearly identical.
    * This fact is supported by the adversarial validation performed above.
* Patrons who use the bank tend to have balances that are over $70,000:
    * This can be observed by the fact that the 25 percentile for estimated salary is 74,440, and the mean is $112,315.
    * This may suggest that this bank is primarily based in savings and investment.
* Patrons are mainly grouped around the ages 32 - 42.
* Most patrons have either one or two products, few have more than two.
* The average number of years patrons have been with the bank are 2.8. 
    * Rarely do patrons stay more than 7 years.
* Credit scores look to be fairly evenly distributed, although we should check to make sure there are no oddities.

# 1.8 - P-Value Testing

While looking at features visually will tell us some interesting information, we can also use p-value testing to see if a feature has a net impact on a simple regression model. This method is controversial in that it likely doesn't provide a correct look at what features are informative. Our null hypothesis is that the feature impacts the target variable of `Exited`. In this case, anything with a p-value greater than 0.05 means we reject that hypothesis, and can potentially flag it for removal. Based on our current examination thus far, and given how few features we have to work with, it is very likely that all our features are informative.

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])

x = add_constant(train_copy[features])
model = OLS(train_copy["Exited"], x).fit()

pvalues = pd.DataFrame(model.pvalues)
pvalues.reset_index(inplace=True)
pvalues.rename(columns={0: "pvalue", "index": "feature"}, inplace=True)
pvalues.style.background_gradient(cmap='YlOrRd')

### Key Observations about P-Value Test

* P-value testing suggests there are no features that are likely candidates for removal.

# 1.9 - Duplicated Rows

As was apparent in past episodes of the Playground, duplicate entries can cause unique challenges to a competition. We should check to see how many duplicate rows exist. In this case, we'll look for duplicates in the training and testing sets, without considering the `id` or `Exited` columns.

In [ ]:
duplicates = train.pivot_table(index=[
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
], aggfunc="size")
unique, counts = np.unique(duplicates, return_counts=True)
value_counts = dict(zip(unique, counts))

if len(unique) == 1:
    print(": There are no duplicated rows in the training set")
else:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

    _ = sns.barplot(x=list(value_counts.keys())[1:], y=list(value_counts.values())[1:], ax=ax)
    _ = ax.set_title("Duplicate Counts in Training Set", fontsize=15)
    _ = ax.set_ylabel("Count")
    _ = ax.set_xlabel("Number of Times Row is Duplicated")
    for p in ax.patches:
        height = p.get_height()
        ax.text(
            x=p.get_x()+(p.get_width()/2),
            y=height,
            s="{:d}".format(int(height)),
            ha="center"
        )

We should also check for duplicates in the testing set:

In [ ]:
duplicates = test.pivot_table(index=[
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
], aggfunc="size")
unique, counts = np.unique(duplicates, return_counts=True)
value_counts = dict(zip(unique, counts))

if len(unique) == 1:
    print(": There are no duplicated rows in the testing set")
else:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

    _ = sns.barplot(x=list(value_counts.keys())[1:], y=list(value_counts.values())[1:], ax=ax)
    _ = ax.set_title("Duplicate Counts in Testing Set", fontsize=15)
    _ = ax.set_ylabel("Count")
    _ = ax.set_xlabel("Number of Times Row is Duplicated")
    for p in ax.patches:
        height = p.get_height()
        ax.text(
            x=p.get_x()+(p.get_width()/2),
            y=height,
            s="{:d}".format(int(height)),
            ha="center"
        )

And finally, we should check for duplicates between the training and testing sets.

In [ ]:
combined = pd.concat([train, test]).reset_index(drop=True)

duplicates = combined.pivot_table(index=[
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
], aggfunc="size")
unique, counts = np.unique(duplicates, return_counts=True)
value_counts = dict(zip(unique, counts))

if len(unique) == 1:
    print(": There are no duplicated rows in the combined set")
else:
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

    _ = sns.barplot(x=list(value_counts.keys())[1:], y=list(value_counts.values())[1:], ax=ax)
    _ = ax.set_title("Duplicate Counts in Training and Testing Sets Combined", fontsize=15)
    _ = ax.set_ylabel("Count")
    _ = ax.set_xlabel("Number of Times Row is Duplicated")
    for p in ax.patches:
        height = p.get_height()
        ax.text(
            x=p.get_x()+(p.get_width()/2),
            y=height,
            s="{:d}".format(int(height)),
            ha="center"
        )

This is an interesting result, since it demonstrates that there is duplicate column overlap between training and testing in the order of over 170 rows. If this is the case, then we may be able to copy the prediction result directly from the training set to the testing set to boost our classifier performance. If we take a closer look at the rows that are duplicated:

In [ ]:
train['origin'] = "train"
test['origin'] = "test"
combined = pd.concat([train, test]).reset_index(drop=True)

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

pd.set_option('display.max_rows', 70)
combined[combined.duplicated(subset=features, keep=False)].sort_values(by=features)

### Key Observations about Duplicated Rows

* There appear to be training samples that we may have a direct target for in the training set:
    * Surname of `Ahern` with a credit score of 745 appears in both the training and testing sets.
    * If we assume the target column of `Exited` is the same for the both, then we can use the training target as our test prediction. 
* There is evidence however, that the duplicate columns may have different mappings to `Exited`:
    * Surname of `Zetticci` with a credit score of 791 appears in the training set twice, with the only difference being the target value of `Exited`.
    * This duplicate outcome is likely to confuse our classifier, since one is a positive example, and the other is negative.
* Depending on how these duplicates were created, if we assume for a moment that these were meant to be distinct rows of data, then we can make the opposite prediction from the training set in the testing set. We can test this empirically by burning a few submissions to see how it impacts score.

# 1.10 - Dimensionality Reduction - UMAP

We may want to explore reducing the dimensionality of the data to see if there is anything interesting. We can use UMAP (Uniform Manifold Approximation and Projection) to reduce the dataset into a fewer number of dimensions. Similar to a PCA, UMAP attempts to reduce the dimensionality of a dataset by utilizing manifold learning. It's primary purpose is to help visualize what a dataset looks like. Ideally, this reduction in dimensionality may also be of benefit when it comes to machine learning tasks, in this case, potentially giving rise to a classifier with better performance than one that attempts to use the entire dataset. We can use a UMAP reduction and use the reduced features in a machine learning regressor directly.

In [ ]:
import umap

umap_train = train.copy()
umap_train = umap_train.iloc[::10, :]

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    umap_train[feature] = le.fit_transform(umap_train[feature])

reducer = umap.UMAP(random_state=2023)
reduced_data = reducer.fit_transform(umap_train[features])
umap_train["2d_x"] = reduced_data[:, 0]
umap_train["2d_y"] = reduced_data[:, 1]

sns.set(style='white', context='notebook')

f, ax = plt.subplots(figsize=(10, 10))

_ = sns.scatterplot(data=umap_train, x="2d_x", y="2d_y", hue="Exited", ax=ax)
_ = ax.set_title('2D Reduced Representation', fontsize=24)

### Key Observations about Dimensionality Reduction

* As we can see from the image, positive and negative samples are fairly intermixed when viewing a reduced representation.
    * There may be pockets of x and y coordinates that provide separation between classes (e.g. x between 10 and 15 with y between 3 and 5).
    * The benefit here may not be so certain as a caveat to this approach is that only every 10th row of data was used to produce the above image.

# 1.11 - Class Imbalance

Finally, we should check to see if there is any significant class skew that may impact our classifier and its performance.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 8))

sns.set_style('darkgrid')
sns.set_palette('Set2')

counts = pd.DataFrame({"Exited": [train["Exited"].value_counts()[0], train["Exited"].value_counts()[1]]})
_ = sns.barplot(x=counts.index, y=counts.Exited, ax=axs[0])
for p in axs[0].patches:
    axs[0].text(x=p.get_x()+(p.get_width()/2), y=p.get_height(), s="{:,d}".format(round(p.get_height())), ha="center")
_ = axs[0].set_title("Class Balance", fontsize=15)
_ = axs[0].set_ylabel("Number of Records", fontsize=15)
_ = axs[0].set_xlabel("Target", fontsize=15)
for label in axs[0].get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')

targets = train["Exited"].unique()
data = [train[(train["Exited"] == target)]["id"].count() for target in targets]
_ = axs[1].pie(
    data, labels=targets,
    autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
    explode=[0.20] * len(data), 
    colors=sns.color_palette("Set2")[0:len(data)],
)
_ = axs[1].set_title("Class Balance", fontsize=15)

### Key Observations about Class Balance

* As we can see, the dataset is heavily skewed to an `Exited` value of 0. 
* Detecting `Exited` values of 1 is likely going to be difficult, as they are rare when compared to the negative class.

# 2 - Feature Exploration

Let's take a closer look at the features we have in our dataset.


# 2.1 - Surname

The `Surname` feature of the dataset provides the last name of the customer. In terms of usefulness, the Spearman correlation suggested that it was positively correlated with bank balance. Balance in turn was correlated with our target variable. A combination of these features may provide us some insights. First, let's look to see how many unique last names we are dealing with.

In [ ]:
print(f": Number of unique surnames = {train['Surname'].nunique():,d}")

This is interesting given there are 165,034 rows of data. This means there is a lot of overlap between last name and number of records. Let's take a look at the top 24 surnames and see if there are any that stick out as having higher correlation to the target.

In [ ]:
fig, axs = plt.subplots(nrows=6, ncols=4, figsize=(15, 25))

sns.set_style('darkgrid')
sns.set_palette('Set2')

top_surnames = [surname for surname in train["Surname"].value_counts().nlargest(24).index]
labels = top_surnames.copy()
axs = axs.flatten()

for ag, surname in enumerate(top_surnames):
    data = [
        train[(train["Exited"] == 0) & (train["Surname"] == surname)]["id"].count(),
        train[(train["Exited"] == 1) & (train["Surname"] == surname)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=label,
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{labels[ag]}", fontsize=15)
    
axs[5].set_axis_off()

As we can see, there are no magic bullets. Some notable surnames such as `Genovese` and `Ch'ang` have slightly higher `Exited` occurrences than their counterparts, while `Onyemauchechukwu` has a much smaller number of `Exited` occurrences. Gradient boosted decision trees may be able to exploit this if we make use of label encoding. 

Strangely though, if we think about surnames and their impact on whether someone leaves the bank, we would expect name to have no impact on the actual result. In other words, in our pie chart, we would expect the different surnames to exit the bank at roughly the same proportions as each other, but clearly this is not the case (for example with `Genovese`). If we examine the surnames closer as well, we find some oddities, such as `H?` and `Hs?`. We may be seeing artifacts of the synthetic data generation scheme at work. What happens if we roll up surnames to first letter only and look at their breakdown of exiting the bank?

In [ ]:
train["Surname_First_Letter"] = train["Surname"].apply(lambda x: x[0])

fig, axs = plt.subplots(nrows=5, ncols=5, figsize=(20, 25))

sns.set_style('darkgrid')
sns.set_palette('Set2')

top_surnames = [surname for surname in train["Surname_First_Letter"].value_counts().index]
labels = top_surnames.copy()
axs = axs.flatten()

for ag, surname in enumerate(top_surnames):
    data = [
        train[(train["Exited"] == 0) & (train["Surname_First_Letter"] == surname)]["id"].count(),
        train[(train["Exited"] == 1) & (train["Surname_First_Letter"] == surname)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=label,
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{labels[ag]}", fontsize=15)
    
axs[5].set_axis_off()

Again, we can see that if the surname starts with `Q`, they are much more likely to leave the bank (however, this can partly be explained by the small numbers of samples in those groups). But if we look at surnames that start with `T` and surnames that start with `C`, we can see a stark difference between those exit numbers. We may want to experiment with first letter surnames as a feature in the classifier to see if it provides lift.

### Key Observations about Surname

* Surname field alone is likely not going to be too helpful in generating lift for our classifier.
* We may be able to combine Surname with other fields to see if there is benefit.
* Different types of categorical encoding may make a difference to our classifier.
* Using only the first letter in the surname as a feature may provide lift.

# 2.2 - CreditScore

The `CreditScore` feature of the data appears to correlate with the patron's real world credit score. According to [Investopedia](https://www.investopedia.com/terms/c/credit_score.asp) a credit score is used as a general guideline to determine whether an individual is a good credit risk (i.e. if they should be given financial credit). The score ranges in values between 300 and 850. Scoring higher means that an individual will likely have a better chance at securing various forms and amounts of credit. Usually those with lower values of credit scores are more likely to make payments late, owe larger amounts, have shorter credit histories, or have too many other lines of credit open.

In terms of relating a credit score to churn, our Spearman correlation showed us that the raw score did not appear to have a strong positive or negative correlation to the target variable. However, there may be ways of segmenting or otherwise modifying the credit score to be more informative. Let's look first at some density estimates to see where the bulk of our credit scores are located.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

_ = sns.kdeplot(train["CreditScore"], shade=True, color="r", ax=axs, label="Credit Score Densities")
_ = axs.set_title("Credit Score Densities", fontsize=15)
_ = axs.set_ylabel("Density")
_ = axs.set_xlabel("Credit Score")

As was indicated on our statistical breakdown above, the bulk of our scores occur around the 650 mark, and tend not to stray too much below 580. This provides us a unique insight based on Investopedia, where credit scores can be categorized into good versus poor risks for credit:

* 300 - 579 = Poor
* 580 - 669 = Fair
* 670 - 739 = Good
* 740 - 799 = Very Good
* 800 - 850 = Excellent

This suggests we may be able to create a new column that segments patrons into different credit ranges, and use that as a feature. Let's look at whether this will be helpful.

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

credit_score_ranges = [(300, 579), (579, 669), (669, 739), (739, 799), (799, 950)]
labels = ["Poor (300-579)", "Fair (580-669)", "Good (670-739)", "Very Good (740-799)", "Excellent (800+)"]
axs = axs.flatten()

for ag, credit_score in enumerate(credit_score_ranges):
    data = [
        train[(train["Exited"] == 0) & (train["CreditScore"] >= credit_score[0]) & (train["CreditScore"] < credit_score[1])]["id"].count(),
        train[(train["Exited"] == 1) & (train["CreditScore"] >= credit_score[0]) & (train["CreditScore"] < credit_score[1])]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=label,
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{labels[ag]}", fontsize=15)
    
axs[5].set_axis_off()

In general, gradient boosting classifiers are likely to already identify this type of data partitioning. The question remains whether this generalized rating is simply too coarse to be of much use. More specifically, if we narrow the boundaries between credit ratings, can we achieve a better class separation?

In [ ]:
def bin_data(series, bin_defs):
    bins = [0 for _ in range(len(bin_defs))]
    total = 0
    for x in series:
        for index, (bin_min, bin_max) in enumerate(bin_defs):
            if x >= bin_min and x < bin_max:
                bins[index] += 1
                total += 1
                break
    return [float(x / total) if total != 0 else 0 for x in bins]

bin_defs = []
labels = []
step_size = 10
for x in range(400, 860, step_size):
    bin_defs.append([x, x+step_size])
    labels.append("{:.2f} - {:.2f}".format(x, x+step_size))
    
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))

sns.set_style('darkgrid')
sns.set_palette('Set2')

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0)]["CreditScore"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1)]["CreditScore"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=ax)
_ = ax.set_title("Credit Score vs Exited", fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("Credit Score")

Again, in terms of classifying based on a credit score, we want to see columns that are very clearly defined as having the majority as being the positive case or negative case. Again, we're not seeing that type of separation. If we more even finger-grained:

In [ ]:
bin_defs = []
labels = []
step_size = 5
for x in range(400, 860, step_size):
    bin_defs.append([x, x+step_size])
    labels.append("{:.2f} - {:.2f}".format(x, x+step_size))
    
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))

sns.set_style('darkgrid')
sns.set_palette('Set2')

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0)]["CreditScore"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1)]["CreditScore"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=ax)
_ = ax.set_title("Credit Score vs Exited", fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("Credit Score")

There simply is not a good slicing that would provide us class separation. We can use some kernel density plots to confirm what we're seeing as well.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["CreditScore"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["CreditScore"], shade=True, color="b", ax=ax, label="No Exit")
_ = ax.set_title("Exited Findings by CreditScore".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

As we can see, there are no places on the plot where we clearly see instances where just the positive class occurring.

### Key Observations about Credit Score

* Credit score as a first order feature, does not appear to provide adequate signal to determine if churn is occurring.

# 2.3 - Geography

The `Geography` feature appears to be categorical in nature, with only three different values. Let's look a their correlation to `Exited`. 

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

geographies = [place for place in train['Geography'].unique()]
axs = axs.flatten()

for ag, geography in enumerate(geographies):
    data = [
        train[(train["Exited"] == 0) & (train["Geography"] == geography)]["id"].count(),
        train[(train["Exited"] == 1) & (train["Geography"] == geography)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=["Did Not Exit", "Exited"],
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{geography}", fontsize=15)
    
axs[3].set_axis_off()

A little bit of insight comes from the fact that `Germany` appears to have a higher number of patrons that exit a bank than others do. Alone this may not be enough of an insight to provide our classifier with any lift, but in combination with other risk factors, it may provide a second-order feature that provides lift.

While our Spearman correlation didn't show strong connection to `Exited`, it did however have a strong correlation to `Balance`, which in turn does have a high connection to `Exited`. Let's take a look for a moment at `Balance` and `Geography`.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))

_ = sns.kdeplot(train[(train["Geography"] == "Germany")]["Balance"], shade=True, color="r", ax=axs, label="Germany")
_ = sns.kdeplot(train[(train["Geography"] == "France")]["Balance"], shade=True, color="g", ax=axs, label="France")
_ = sns.kdeplot(train[(train["Geography"] == "Spain")]["Balance"], shade=True, color="b", ax=axs, label="Spain")
_ = axs.set_title("Balance Densities by Geography", fontsize=15)
_ = axs.set_ylabel("Density")
_ = axs.set_xlabel("Balance")
_ = axs.legend()

As we can see, there is a very noticeable difference in terms of balance distributions between various geographies, with `Germany` being the one geography that has the bulk of it's balances sitting at roughly $125,000. Again, this doesn't help us with the `Exited` value, but it does show us there are differences between geographic locations.

### Key Insights about Geography

* The breakdown of `Exited` by geography didn't provide a clear indicator for our classifier.
    * However, we may be able to use `Germany` as a risk factor calculation, since it is clear that people leaving occurs more in Germany than other geographic locations.

# 2.4 - Gender

Our Spearman correlation showed us that there was a weak negative correlation between the `Gender` feature and `Exited`. Let's take a look at gender versus our target to see if there is anything interesting.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

genders = [place for place in train['Gender'].unique()]
axs = axs.flatten()

for ag, gender in enumerate(genders):
    data = [
        train[(train["Exited"] == 0) & (train["Gender"] == gender)]["id"].count(),
        train[(train["Exited"] == 1) & (train["Gender"] == gender)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=["Did Not Exit", "Exited"],
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{gender}", fontsize=15)

As we can see, patrons who are `Male` are much more likely to stay with the bank when compared with those that are `Female`. The question is whether there are even clearer indications of when `Male` patrons may leave. Let's take a look at `Gender` combined with `Geography`. Specifically, let's look at `Male` patrons, since they have a different likelihood of exiting.

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

geographies = [place for place in train['Geography'].unique()]
axs = axs.flatten()

for ag, geography in enumerate(geographies):
    data = [
        train[(train["Exited"] == 0) & (train["Geography"] == geography) & (train["Gender"] == "Male")]["id"].count(),
        train[(train["Exited"] == 1) & (train["Geography"] == geography) & (train["Gender"] == "Male")]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=["Did Not Exit", "Exited"],
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"Male Exited Counts in {geography}", fontsize=15)
    
axs[3].set_axis_off()

Again, we see lower numbers of males exiting from France and Spain. Again, this is not a strong enough indicator on its own, but could be combined into a _mitigating factors_ feature that counts the number of items that make it more _likely_ that a patron will stay with the bank. Again, models such as Gradient Boosting Decision Tree approaches are very likely to find this connection without any further input or processing. Let's take a look for a moment at balances.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))

_ = sns.kdeplot(train[(train["Gender"] == "Male")]["Balance"], shade=True, color="r", ax=axs, label="Male")
_ = sns.kdeplot(train[(train["Gender"] == "Female")]["Balance"], shade=True, color="b", ax=axs, label="Female")
_ = axs.set_title("Balance Densities by Gender", fontsize=15)
_ = axs.set_ylabel("Density")
_ = axs.set_xlabel("Balance")
_ = axs.legend()

We see nearly identical balance densities. If we break this down further by geographical location:

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

_ = sns.kdeplot(train[(train["Gender"] == "Male") & (train["Geography"] == "Germany")]["Balance"], shade=True, color="r", ax=axs[0], label="Male")
_ = sns.kdeplot(train[(train["Gender"] == "Female") & (train["Geography"] == "Germany")]["Balance"], shade=True, color="b", ax=axs[0], label="Female")
_ = axs[0].set_title("Balance Densities by Gender in Germany", fontsize=15)
_ = axs[0].set_ylabel("Density")
_ = axs[0].set_xlabel("")
_ = axs[0].legend()

_ = sns.kdeplot(train[(train["Gender"] == "Male") & (train["Geography"] == "France")]["Balance"], shade=True, color="r", ax=axs[1], label="Male")
_ = sns.kdeplot(train[(train["Gender"] == "Female") & (train["Geography"] == "France")]["Balance"], shade=True, color="b", ax=axs[1], label="Female")
_ = axs[1].set_title("Balance Densities by Gender in France", fontsize=15)
_ = axs[1].set_ylabel("Density")
_ = axs[1].set_xlabel("")
_ = axs[1].legend()

_ = sns.kdeplot(train[(train["Gender"] == "Male") & (train["Geography"] == "Spain")]["Balance"], shade=True, color="r", ax=axs[2], label="Male")
_ = sns.kdeplot(train[(train["Gender"] == "Female") & (train["Geography"] == "Spain")]["Balance"], shade=True, color="b", ax=axs[2], label="Female")
_ = axs[2].set_title("Balance Densities by Gender in Spain", fontsize=15)
_ = axs[2].set_ylabel("Density")
_ = axs[2].set_xlabel("")
_ = axs[2].legend()

Again, balance densities are very similar between genders regardless of geography.

### Key Observations about Gender

* Males are far less likely to exit the bank.
    * This holds true in both `Spain` and `France`.
    * Males in `Germany` are much more likely to exit when compared to other geographies.
* Males and females have very similar balances.

# 2.5 - Age

First of all, let's take a look at just the `Age` feature alone to see whether we have class separation.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["Age"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["Age"], shade=True, color="b", ax=ax, label="No Exit")
_ = ax.set_title("Exited Findings by Age".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

We have two very distinct age densities associated with `Exited`. As we can see, those aged 25-35 are likely to remain with the bank, while those aged 40 - 65 are more likely to exit. This finding is consistent with our Spearman correlation, which suggested that higher ages were more strongly correlated with the target. As a first-order feature, this is good news, since it suggests that one of the best indicators of churn is age. Much like our `Geography` column, we can also use this as an absolute value for a risk factor. If age is greater than 40, an additional risk factor exists.

The question is whether we can gain additional lift by enhancing the feature with other continuous columns. A good candidate is the number of products that the user has, since our Spearman correlation said that the number of products was negatively correlated with the target variable (the more products a patron has, the less likely they are to leave). If we exaggerate age by number of products, we may be able to gain class separation.

In [ ]:
train["Age_NumOfProducts"] =  train["Age"] ** train["NumOfProducts"]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["Age_NumOfProducts"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["Age_NumOfProducts"], shade=True, color="b", ax=ax, label="No Exit")
_ = ax.set_title("Exited Findings by Age ** NumOfProducts".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
_ = ax.set_xlim(0, 0.045e7)
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

In the plot above, we have zoomed in substantially. We can see that we do have quite a large class separation between those that exited and those that didn't. What the plot isn't showing is the long tail for the `No Exit` grouping. This particular feature combination may provide additional lift to our classifier.

### Key Observations about Age

* Age is a strong indicator for our target.
* We may be able to create an additional risk factor based on age.
* Age combined with number of products may provide lift as an additional feature.

# 2.6 - Tenure

Tenure refers to how long a patron has been with the institution. Let's take a look at raw tenure numbers and see how they pan out.

In [ ]:
bin_defs = []
labels = []
step_size = 1
for x in range(0, 10, step_size):
    bin_defs.append([x, x+step_size])
    labels.append(f"{x:d}")
    
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))

sns.set_style('darkgrid')
sns.set_palette('Set2')

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0)]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1)]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=ax)
_ = ax.set_title("Tenure vs Exited", fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("Tenure (Years)")

As we can see, there isn't really a difference in raw tenure years. There are however, a number of ways to look at tenure statistics. First, let's see if there is a difference in `Gender`.

In [ ]:
bin_defs = []
labels = []
step_size = 1
for x in range(0, 10, step_size):
    bin_defs.append([x, x+step_size])
    labels.append(f"{x:d}")
    
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')
sns.set_palette('Set2')

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0) & (train["Gender"] == "Male")]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1) & (train["Gender"] == "Male")]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=axs[0])
_ = axs[0].set_title("Tenure vs Exited For Male Population", fontsize=15)
_ = axs[0].set_ylabel("")
_ = axs[0].set_xlabel("")

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0) & (train["Gender"] == "Female")]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1) & (train["Gender"] == "Female")]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=axs[1])
_ = axs[1].set_title("Tenure vs Exited For Female Population", fontsize=15)
_ = axs[1].set_ylabel("")
_ = axs[1].set_xlabel("")

No magic bullets. Let's look at tenure based on geography.

In [ ]:
bin_defs = []
labels = []
step_size = 1
for x in range(0, 10, step_size):
    bin_defs.append([x, x+step_size])
    labels.append(f"{x:d}")
    
fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0) & (train["Geography"] == "Germany")]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1) & (train["Geography"] == "Germany")]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=axs[0])
_ = axs[0].set_title("Tenure vs Exited in Germany", fontsize=15)
_ = axs[0].set_ylabel("")
_ = axs[0].set_xlabel("")

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0) & (train["Geography"] == "Spain")]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1) & (train["Geography"] == "Spain")]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=axs[1])
_ = axs[1].set_title("Tenure vs Exited in Spain", fontsize=15)
_ = axs[1].set_ylabel("")
_ = axs[1].set_xlabel("")

tdf = pd.DataFrame(
    {
        'Label': labels,
        'Did Not Exit': bin_data(train[(train["Exited"] == 0) & (train["Geography"] == "France")]["Tenure"], bin_defs),
        'Exited': bin_data(train[(train["Exited"] == 1) & (train["Geography"] == "France")]["Tenure"], bin_defs),
    }
)

_ = tdf.set_index('Label').plot(kind='bar', stacked=True, color=["blue", "red"], ax=axs[2])
_ = axs[2].set_title("Tenure vs Exited in France", fontsize=15)
_ = axs[2].set_ylabel("")
_ = axs[2].set_xlabel("")

Again, no particular tenure year based on geography appears to stand out. Let's see if age plays a factor with tenure.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="Tenure", y="Age", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited Age vs Tenure", fontsize=15)
_ = ax.set_ylabel("Age")
_ = ax.set_xlabel("Tenure")

Again, no significant difference between tenure groups and the relative ages when people exit. 

# 2.8 - Balance

To start with, let's cut right to a density plot and see if the `Balance` feature shows any clear separation between those who exited, and those who didn't. Given the high correlation between `Balance` and `Exited`, we expect to see some separation.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["Balance"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["Balance"], shade=True, color="b", ax=ax, label="Did Not Exit")
_ = ax.set_title("Density of Balances".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
_ = ax.set_xlim(0, 250000)
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

Interestingly, it appears that having a $0 balance would be a mitigating factor in regards to whether a patron exits. This does raise an interesting point: why be a patron when you have no balance? Perhaps balances vs number of products will tell us.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="NumOfProducts", y="Balance", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited Balance vs NumOfProducts", fontsize=15)
_ = ax.set_ylabel("Balance")
_ = ax.set_xlabel("NumOfProducts")

This is starting to look more interesting. While having 1 or 4 products doesn't help us separate out those that exit from those that don't, the interesting cases occur when there are 2 or 3 products. To summarize:

* 2 products and $0 balance means a user is far less likely to exit.
* 3 products and a $0 balance means a user is far less likely to exit.
* 3 products and a balance means the user is far more likely to exit.

We can use this information to our advantage as more refined mitigating or risk factors.

# 2.9 - NumOfProducts

_ In progress..._

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["NumOfProducts"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["NumOfProducts"], shade=True, color="b", ax=ax, label="No Exit")
_ = ax.set_title("Exited Findings by NumOfProducts".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="NumOfProducts", y="Age", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited Age vs NumOfProducts", fontsize=15)
_ = ax.set_ylabel("Age")
_ = ax.set_xlabel("NumOfProducts")

# 2.10 - HasCrCard

_In progress..._

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

has_crcard_types = [place for place in train['HasCrCard'].unique()]
axs = axs.flatten()

for ag, has_crcard in enumerate(has_crcard_types):
    data = [
        train[(train["Exited"] == 0) & (train["HasCrCard"] == has_crcard)]["id"].count(),
        train[(train["Exited"] == 1) & (train["HasCrCard"] == has_crcard)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=["Did Not Exit", "Exited"],
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{'Has Credit Card' if has_crcard == 1 else 'No Credit Card'}", fontsize=15)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="NumOfProducts", y="HasCrCard", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited HasCrCard vs NumOfProducts", fontsize=15)
_ = ax.set_ylabel("HasCrCard")
_ = ax.set_xlabel("NumOfProducts")

# 2.11 - IsActiveMember

_In progress..._

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))

sns.set_style('darkgrid')
sns.set_palette('Set2')

is_active_status_types = [place for place in train['IsActiveMember'].unique()]
axs = axs.flatten()

for ag, is_active in enumerate(is_active_status_types):
    data = [
        train[(train["Exited"] == 0) & (train["IsActiveMember"] == is_active)]["id"].count(),
        train[(train["Exited"] == 1) & (train["IsActiveMember"] == is_active)]["id"].count()
    ]

    label = ["Did Not Exit", "Exited"]
    _ = axs[ag].pie(
        data, labels=["Did Not Exit", "Exited"],
        autopct=lambda x: "{:,.0f} = {:.2f}%".format(x * sum(data)/100, x),
        explode=[0.05] * 2, 
        pctdistance=0.5, 
        colors=sns.color_palette("Set2")[0:2],
    )
    _ = axs[ag].set_title(f"{'Active' if is_active == 1 else 'Not Active'}", fontsize=15)

# 2.12 - EstimatedSalary

_In progress..._

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["EstimatedSalary"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["EstimatedSalary"], shade=True, color="b", ax=ax, label="Did Not Exit")
_ = ax.set_title("Density of EstimatedSalary".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
_ = ax.set_xlim(0, 250000)
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="NumOfProducts", y="EstimatedSalary", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited EstimatedSalary vs NumOfProducts", fontsize=15)
_ = ax.set_ylabel("EstimatedSalary")
_ = ax.set_xlabel("NumOfProducts")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

sns.set_style('darkgrid')

_ = sns.violinplot(data=train, x="HasCrCard", y="EstimatedSalary", hue="Exited", split=True, inner="quart")
_ = ax.set_title("Exited EstimatedSalary vs HasCrCard", fontsize=15)
_ = ax.set_ylabel("EstimatedSalary")
_ = ax.set_xlabel("HasCrCard")

In [ ]:
train["Future_Worth"] = train["EstimatedSalary"] + train["Balance"]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))

sns.set_style('darkgrid')

_ = sns.kdeplot(train[(train["Exited"] == 1)]["Future_Worth"], shade=True, color="r", ax=ax, label="Exited")
_ = sns.kdeplot(train[(train["Exited"] == 0)]["Future_Worth"], shade=True, color="b", ax=ax, label="Did Not Exit")
_ = ax.set_title("Density of Future_Worth".format(feature), fontsize=15)
_ = ax.set_ylabel("")
_ = ax.set_xlabel("")
handles, labels = ax.get_legend_handles_labels()
_ = ax.legend(handles=handles[0:2], labels=labels[0:2], title="")

# 3 - Model Comparison

Let's take a look at a few basic models, and see how well they perform with different types of features.

# 3.1 - Vanilla LightGBM

Based on the mix of categorical data, LightGBM is a good model to start with, since we can easily tell it what columns are categorical. We'll label encode the categorical features `Surname`, `Geography` and `Gender`.

In [ ]:
from lightgbm import LGBMClassifier
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_baseline = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_baseline:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
from sklearn.calibration import CalibrationDisplay

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

With very little tuning, the classifier does very well, scoring an AUC ROC of 0.89121. The calibration of our predictions looks good too, with predictions fairly spread out along the probability spectrum, although there is some clumping toward the negative class. 

# 3.2 - LightGBM Categorical

LightGBM has built-in support for dealing with categorical features, but has to be told which columns are to be interpreted as categorical. Features must be integer encoded. There are smoothing options to deal with categorical features with high cardinality, but we will ignore those for now and use default settings.

In [ ]:
features = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
        categorical_features = [0, 1, 2],
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_category = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_category:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

As we can see, we achieve a slightly less performant result, probably due to the high cardinality of the `Surname` category.

# 3.3 - Feature: First Letter Surname Group

As discussed at the start of our EDA, using just the first letter of the surname may provide additional information to a classifier. Let's try that as a feature.

In [ ]:
features = [
    'Surname', 'CreditScore', 'Geography', 'Gender',
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()
train_copy["Surname_First_Letter"] = train_copy["Surname"].apply(lambda x: x[0])
features.append("Surname_First_Letter")

for feature in ["Surname", "Geography", "Gender", "Surname_First_Letter"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_first_letter_surname = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_first_letter_surname:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

# 3.4 - Feature: Age ** NumOfProducts

As discussed in our EDA, our age amplified by the number of products the patron has may provide additional lift. We should test this out as a feature.

In [ ]:
features = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

train_copy["Age_NumOfProducts"] = train_copy["Age"] ** train_copy["NumOfProducts"]
features.append("Age_NumOfProducts")

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])
    
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_age_prods = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_age_prods:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

# 3.5 - Feature: Risk Factor Count

One of the items discussed throughout the EDA was the notion of risk factors. Of these, we came up with the following:

* `Age` > 40
* `Geography` = Germany
* `NumOfProducts` > 2

With that in mind, we can create a risk factor feature that sums up the total number of risk factors.

In [ ]:
features = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

train_copy["Risk_Geography"] = train_copy["Geography"].apply(lambda x: 1 if x == "Germany" else 0)
train_copy["Risk_Age"] = train_copy["Age"].apply(lambda x: 1 if x >= 40 else 0)
train_copy["Risk_NumOfProducts"] = train_copy["NumOfProducts"].apply(lambda x: 1 if x > 2 else 0)
train_copy["RiskFactors"] = train_copy["Risk_Geography"] + train_copy["Risk_Age"] + train_copy["Risk_NumOfProducts"]
features.append("RiskFactors")

for feature in ["Surname", "Geography", "Gender"]:
    le = LabelEncoder()
    train_copy[feature] = le.fit_transform(train_copy[feature])
    
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_risk_factors = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_risk_factors:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

# 3.6 - Target Encoding

We have a number of encoding options when it comes to encoding our categorical data into a form our classifiers can use. While label encoding is quick, we have other options such as target encoding. With target encoding, we calculate the mean value of the categorical value with respect to the target. In other words, we calculate the prior probability of the categorical value being associated with `Exited`. This method does have a drawback: it leaks information about our target variable into the features themselves. However, given that it is computationally inexpensive, we should explore it as an option.

In [ ]:
from category_encoders.target_encoder import TargetEncoder

features = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

for feature in ["Surname", "Geography", "Gender"]:
    te = TargetEncoder()
    train_copy[feature] = te.fit_transform(train_copy[feature], train_copy["Exited"])
    
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_target_encode = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_target_encode:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

# 3.7 - All Beneficial Features

With all our beneficial features turned on, let's run a check to ensure we see lift.

In [ ]:
features = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

train_copy = train.copy()

train_copy["Surname_First_Letter"] = train_copy["Surname"].apply(lambda x: x[0])
features.append("Surname_First_Letter")

for feature in ["Surname", "Geography", "Gender", "Surname_First_Letter"]:
    te = TargetEncoder()
    train_copy[feature] = te.fit_transform(train_copy[feature], train_copy["Exited"])

train_copy["Risk_Geography"] = train_copy["Geography"].apply(lambda x: 1 if x == "Germany" else 0)
train_copy["Risk_Age"] = train_copy["Age"].apply(lambda x: 1 if x >= 40 else 0)
train_copy["Risk_NumOfProducts"] = train_copy["NumOfProducts"].apply(lambda x: 1 if x >= 2 else 0)
train_copy["RiskFactors"] = train_copy["Risk_Geography"] + train_copy["Risk_Age"] + train_copy["Risk_NumOfProducts"]
features.append("RiskFactors")

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=2023, shuffle=True)
train_oof_preds = np.zeros((train.shape[0],))
scores = []

for fold, (train_index, test_index) in enumerate(skf.split(train_copy, train_copy["Exited"])):
    print(f"-------> Fold {fold+1} <--------")
    x_train, x_valid = pd.DataFrame(train_copy.iloc[train_index]), pd.DataFrame(train_copy.iloc[test_index])
    y_train, y_valid = train_copy["Exited"].iloc[train_index], train_copy["Exited"].iloc[test_index]
    
    x_train_features = pd.DataFrame(x_train[features])
    x_valid_features = pd.DataFrame(x_valid[features])

    model = LGBMClassifier(
        random_state=2023,
        objective="binary",
        metric="auc",
        n_jobs=-1,
        n_estimators=5000,
        verbose=-1,    
    )
    model.fit(
        x_train_features[features], 
        y_train,
        eval_set=[(x_valid_features[features], y_valid)],
        callbacks=[
            early_stopping(50, verbose=False),
            log_evaluation(5000),
        ]
    )
    oof_preds = model.predict_proba(x_valid_features[features])[:,1]
    train_oof_preds[test_index] = oof_preds
    score = roc_auc_score(y_valid, oof_preds)
    scores.append(score)
    print(f": AUC ROC = {score:.5f}")
    
auc_all_good = roc_auc_score(train["Exited"], train_oof_preds)
print("--> Overall results for out of fold predictions")
print(f": AUC ROC = {auc_all_good:.5f}")

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

data = pd.DataFrame({"Fold": [x + 1 for x in range(n_folds)], "AUC ROC": scores})
_ = sns.lineplot(x="Fold", y="AUC ROC", data=data, ax=ax)
_ = ax.set_title("AUC ROC per Fold", fontsize=15)
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_xlabel("Fold #")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 8))

CalibrationDisplay.from_predictions(train_copy["Exited"], train_oof_preds, n_bins=30, strategy='quantile', ax=ax)
_ = ax.set_title("Prediction Calibration")

# 3.x - Model Comparisons

Let's compare how the different approaches perform.

In [ ]:
bar, ax = plt.subplots(figsize=(20, 10))
ax = sns.barplot(
    x=["Baseline", "Categorical On", "First Letter Surname", "Age ** Products", "Risk Factors", "Target Encoding", "All Good"],
    y=[
        auc_baseline,
        auc_category,
        auc_first_letter_surname,
        auc_age_prods,
        auc_risk_factors,
        auc_target_encode,
        auc_all_good,
    ]
)
_ = ax.axhline(y=auc_baseline, color='r', linestyle='--')
_ = ax.set_title("AUC ROC (Higher is Better)", fontsize=15)
_ = ax.set_xlabel("")
_ = ax.set_ylabel("AUC ROC")
_ = ax.set_ylim([0.89, 0.90])
for p in ax.patches:
    height = p.get_height()
    ax.text(
        x=p.get_x()+(p.get_width()/2),
        y=height,
        s="{:.5f}".format(height),
        ha="center"
    )

# 4 - Conclusions

There are some interesting points we discovered in the datasets:

* The dataset is highly skewed to the negative class.
* With a large number of samples to learn from, our results may be more stable between local CV, public LB, and private LB. 
* No null values appear in the training or testing datasets.
* Duplicate columns exist in the dataset, and identical entries have different mappings to `Exited`.
* Adversarial validation suggests that the training dataset and the testing dataset are very similar.
* Adversarial validation suggests there are some easy-to-spot differences between the original dataset and the competition dataset.
    * Caution should be used when mixing data.
    * The `Surname` categorical column appears to be a distinguishing factor between datasets.
* P-value testing suggests there are no features that are likely candidates for removal.
* For the `Surname` feature:
    * Different types of categorical encoding may make a difference to our classifier.
    * Using only the first letter in the surname as a feature may provide lift.
* For the `Geography` feature:
    * People leaving occurs more in Germany than other geographic locations - this may be useful to generate a _risk factor_ feature.
* For the `Gender` feature:
    * Males are far less likely to exit the bank.
    * This holds true in both `Spain` and `France`.
    * Males in `Germany` are much more likely to exit when compared to other geographies.
    * The `Gender` feature in `Spain` and `France` may be useful to generate a _mitigating factors_ feature.
* For the `Age` feature:
    * Age is a strong indicator for our target.
    * We may be able to create an additional risk factor based on age, as ages over 40 are more likely to exit.
    * Age combined with number of products may provide lift as an additional feature.


## More to Come...

In [ ]:
features_test = [
    'Surname', 'Geography', 'Gender', 'CreditScore', 
    'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
    'IsActiveMember', 'EstimatedSalary'
]

# Copy the test data
test_copy = test.copy()

# Target encode categorical features for test data
for feature in ["Surname", "Geography", "Gender", "Surname_First_Letter"]:
    te = TargetEncoder()
    test_copy[feature] = te.fit_transform(test_copy[feature], train_copy["Exited"])

# Additional feature engineering for test data
test_copy["Risk_Geography"] = test_copy["Geography"].apply(lambda x: 1 if x == "Germany" else 0)
test_copy["Risk_Age"] = test_copy["Age"].apply(lambda x: 1 if x >= 40 else 0)
test_copy["Risk_NumOfProducts"] = test_copy["NumOfProducts"].apply(lambda x: 1 if x >= 2 else 0)
test_copy["RiskFactors"] = test_copy["Risk_Geography"] + test_copy["Risk_Age"] + test_copy["Risk_NumOfProducts"]
features_test.append("RiskFactors")

# Ensure the test data contains all the required features
test_features = pd.DataFrame(test_copy[features_test])

# Predict on the test set
preds_test = model.predict_proba(test_features)[:, 1]

# Save test predictions to file
output = pd.DataFrame({'id': test.index, 'Exited': preds_test})
output.to_csv('submission.csv', index=False)